# Spark SQL

Spark SQL:  
* Spark SQL is the engine on which structured API's are built  
* Can read and write data in a variety of formats  
* Can query data using JDBC/ODBC connectors  
* Provides a programmatic interface to interact with structured data stored as tables or views in a database from a spark application    
* Provides an interactive shell to write sql queries      
![Spark SQL](/Images/spark-sql.png)

## Using Spark SQL

* use `sql()` method on `SparkSession` instance
* `spark.sql("select * from table")`
* The output of spark.sql will be a DataFrame

In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder         
         .getOrCreate())

23/05/06 09:24:46 WARN Utils: Your hostname, thulasiram resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp0s20f3)
23/05/06 09:24:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/06 09:24:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path_to_data = '/home/thulasiram/personal/data_engineering/data/car_data.csv'

In [3]:
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(path_to_data))

In [4]:
df.show(5)

23/05/06 09:24:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , make, fuel_type, aspiration, num_of_doors, body_style, drive_wheels, engine_location, wheel_base, length, width, height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg, price
 Schema: _c0, make, fuel_type, aspiration, num_of_doors, body_style, drive_wheels, engine_location, wheel_base, length, width, height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg, price
Expected: _c0 but found: 
CSV file: file:///home/thulasiram/personal/data_engineering/data/car_data.csv
+---+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+--------+-----------+-----+
|_c0|      

In [5]:
df.createGlobalTempView("cars_tbl")

In [6]:
tables = spark.catalog.listTables("global_temp")
for table in tables:
    print(table.name)

cars_tbl


In [7]:
spark.sql("""SELECT make, fuel_type, body_style
            FROM global_temp.cars_tbl
            limit 5""").show()

+-----------+---------+-----------+
|       make|fuel_type| body_style|
+-----------+---------+-----------+
|alfa-romero|      gas|convertible|
|alfa-romero|      gas|convertible|
|alfa-romero|      gas|  hatchback|
|       audi|      gas|      sedan|
|       audi|      gas|      sedan|
+-----------+---------+-----------+



In [8]:
spark.sql("""SELECT body_style, count(*) as body_style_count
            FROM global_temp.cars_tbl
            group by body_style
            order by body_style_count desc""").show()

+-----------+----------------+
| body_style|body_style_count|
+-----------+----------------+
|      sedan|              96|
|  hatchback|              70|
|      wagon|              25|
|    hardtop|               8|
|convertible|               6|
+-----------+----------------+



* Using `createGlobalTempView()` method, the table is registered as a global temporary view
* Global temporary views are tied to a database called `global_temp`
* These are cross-session which means that we can access the global temporary view from any SparkSession within the same application

### Managed Vs Unmanaged Tables

* For managed table, Spark manages both Metadata and data in the file store
* For an unmanaged table, spark only manages the metadata (we need to manage the data ourselves)
* Managed table - 'DROP TABLE table_name' deletes both the metadata and the data
* Unmanaged table - 'DROP TABLE IF EXISTS table_name' deletes only the metadata

In [9]:
spark.sql("CREATE DATABASE cars")
spark.sql("USE cars")

DataFrame[]

#### Creating a Managed table

##### Using SQL 

In [14]:
spark.conf.set('spark.sql.legacy.createHiveTableByDefault', False)

In [15]:
spark.sql("CREATE TABLE managed_cars (make STRING, fuel_type STRING,body_style STRING, wheel_base FLOAT, length FLOAT, width FLOAT, height FLOAT)")

DataFrame[]

In [16]:
spark.catalog.listTables()

[Table(name='managed_cars', database='cars', description=None, tableType='MANAGED', isTemporary=False)]

##### Using DataFrame API

In [20]:
df.write.saveAsTable("managed_df_api_cars")

23/05/06 09:52:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , make, fuel_type, aspiration, num_of_doors, body_style, drive_wheels, engine_location, wheel_base, length, width, height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg, price
 Schema: _c0, make, fuel_type, aspiration, num_of_doors, body_style, drive_wheels, engine_location, wheel_base, length, width, height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg, price
Expected: _c0 but found: 
CSV file: file:///home/thulasiram/personal/data_engineering/data/car_data.csv


In [21]:
spark.sql("select * from managed_df_api_cars limit 3").show()

+---+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+--------+-----------+-----+
|_c0|       make|fuel_type|aspiration|num_of_doors| body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|peak_rpm|city_mpg|highway_mpg|price|
+---+-----------+---------+----------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+--------+-----------+-----+
|  1|alfa-romero|      gas|       std|         two|convertible|         rwd|          front|      88.6| 168.8| 64.1|  48.8|       2548|       dohc|            four|        130|       mpfi|              9.0|       111|    5000|      21|   

### Creating Unmanaged table

* we can create unmanaged tables from our own data sources which is accessible to our spark application

##### Creating table from a CSV file using SQL

In [22]:
spark.sql("""CREATE TABLE unmanaged_cars (make STRING, fuel_type STRING,body_style STRING, wheel_base FLOAT, length FLOAT, width FLOAT, height FLOAT)
        USING csv OPTIONS(PATH '/home/thulasiram/personal/data_engineering/data/car_data.csv')""")

DataFrame[]

##### Creating table with DataFrame API

In [24]:
(df
 .write
 .option("path", "/home/thulasiram/personal/data_engineering/data/unmanaged_car_data.csv")
 .saveAsTable("unmanaged_df_api_cars"))

23/05/06 10:02:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , make, fuel_type, aspiration, num_of_doors, body_style, drive_wheels, engine_location, wheel_base, length, width, height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg, price
 Schema: _c0, make, fuel_type, aspiration, num_of_doors, body_style, drive_wheels, engine_location, wheel_base, length, width, height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg, price
Expected: _c0 but found: 
CSV file: file:///home/thulasiram/personal/data_engineering/data/car_data.csv


### Creating Views

* We can create views on top of existing tables
* It can be global (visible across all sparkSession on a cluter) or session based
* Views disappear after sparkSession is closed

In [32]:
df_hatchback = spark.sql("SELECT * FROM managed_df_api_cars WHERE body_style = 'hatchback'")

In [33]:
# Creating a temporary and global temporary view
df_hatchback.createOrReplaceGlobalTempView("cars_tbl_hatchback_gtemp_view")
df_hatchback.createOrReplaceTempView("cars_tbl_hatchback_temp_view")

In [37]:
spark.sql("SELECT * FROM global_temp.cars_tbl_hatchback_gtemp_view limit 5").show()

+---+-----------+---------+----------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+--------+-----------+-----+
|_c0|       make|fuel_type|aspiration|num_of_doors|body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|peak_rpm|city_mpg|highway_mpg|price|
+---+-----------+---------+----------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+--------+-----------+-----+
|  3|alfa-romero|      gas|       std|         two| hatchback|         rwd|          front|      94.5| 171.2| 65.5|  52.4|       2823|       ohcv|             six|        152|       mpfi|              9.0|       154|    5000|      19|       

### Dropping Views

In [41]:
spark.catalog.dropGlobalTempView("cars_tbl_hatchback_gtemp_view")
spark.catalog.dropTempView("cars_tbl_hatchback_temp_view")

True

### Viewing the Metadata

In [42]:
spark.catalog.listDatabases()

[Database(name='cars', description='', locationUri='file:/home/thulasiram/personal/data_engineering/spark/pyspark/spark-warehouse/cars.db'),
 Database(name='default', description='default database', locationUri='file:/home/thulasiram/personal/data_engineering/spark/pyspark/spark-warehouse')]

In [43]:
spark.catalog.listTables()

[Table(name='managed_cars', database='cars', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='managed_df_api_cars', database='cars', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='unmanaged_cars', database='cars', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='unmanaged_df_api_cars', database='cars', description=None, tableType='EXTERNAL', isTemporary=False)]

In [44]:
spark.catalog.listColumns("managed_df_api_cars")

[Column(name='_c0', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='make', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='fuel_type', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='aspiration', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='num_of_doors', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='body_style', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='drive_wheels', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='engine_location', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='wheel_base', description=None, dataType='double', nullable=True, isPartiti

### Summary

* Create managed and unmanaged tables using spark SQL and DataFrame API
* use `spark.sql` to issue queries on spark sql tables or views
* use `catalog` to inspect the metadata